In [1]:
import wandb
import random
# start a new wandb run to track this script

base_dir = "DukePred_Transfer_Result"

raw_data_dir = "DukePred_Transfer_Result/S"
state_dict_1st_UNet = "../data/synthetic/checkpoint_epoch58.pth"
state_dict_2nd_UNet = "../data/synthetic/checkpoint_epoch112.pth"

SS_NUM = 200
USE_WANDB = False
SHOW = True
if USE_WANDB:
    wandb.init(
        # set the wandb project where this run will be logged
        project="Test_Result",     
        # track hyperparameters and run metadata
        config={
            "Version": "Michael's",
            "state_dict_1st_UNet" :state_dict_1st_UNet,
            "state_dict_2nd_UNet" : state_dict_2nd_UNet,
            "SS_NUM": SS_NUM
        }
    )

    raw_data_at = wandb.Artifact(artifact_dir, type="raw_data")
    raw_data_at.add_dir(artifact_dir)
    

In [2]:
"""rmse using transfer UNet (cascade)
current issue: need to change a number in .forward of the UNet code when changing from 
UNet with isotropic cm to UNet with directional cm since the number of input channels 
are different. Temporary solution: run iso_UNet, save the output image, change the .forward 
function's number (2 to 3 or the other way round), restart kernel, run directional UNet. 
"""

"""
The above comment no longer applies
"""

"""
This is code for plotting the coverage maps as predicted by the isotropic and directional U-Nets.
"""

import logging
import os
import itertools

import math

import sys


from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

import numpy as np

from scipy import ndimage, datasets
from scipy.constants import speed_of_light

import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

import torch
from torch.utils.data import DataLoader, random_split
import torch.nn.functional as F
from torchvision import transforms


from unet.unet_model_rt import UNet
from utils.utils import plot_img_and_mask

from CBRSUtils import CBRSUtils as cu

import random

from pathlib import Path



import pandas as pd
b_map_path_dict = {'C': "/home/yl826/3DPathLoss/nc_raytracing/POC/res_plane_duke_PCI6_7/Bl_building_npy/0_c8681783-aa00-46e7-92b2-ca15c5beca67.npy",
                  'D': "/home/yl826/3DPathLoss/nc_raytracing/POC/res_plane_duke_PCI6_7/Bl_building_npy/0_c8681783-aa00-46e7-92b2-ca15c5beca67.npy",
                  'A': "/home/yl826/3DPathLoss/nc_raytracing/POC/res_plane_duke_PCI6_7/Bl_building_npy/0_c8681783-aa00-46e7-92b2-ca15c5beca67.npy",
                  'B': "/home/yl826/3DPathLoss/nc_raytracing/POC/res_plane_duke_PCI6_7/Bl_building_npy/0_c8681783-aa00-46e7-92b2-ca15c5beca67.npy",
                  'E': "/home/yl826/3DPathLoss/nc_raytracing/POC/0_78d03890-be26-47f8-8b81-8485f0d19a83.npy",
                  'F': "/home/yl826/3DPathLoss/nc_raytracing/POC/0_78d03890-be26-47f8-8b81-8485f0d19a83.npy"}
name_dict = {'C': "0_c8681783-aa00-46e7-92b2-ca15c5beca67.npy",
                  'D': "0_c8681783-aa00-46e7-92b2-ca15c5beca67.npy",
                  'A': "0_c8681783-aa00-46e7-92b2-ca15c5beca67.npy",
                  'B': "0_c8681783-aa00-46e7-92b2-ca15c5beca67.npy",
                  'E': "0_78d03890-be26-47f8-8b81-8485f0d19a83.npy",
                  'F': "0_78d03890-be26-47f8-8b81-8485f0d19a83.npy"}

sionna_cm_base = '/home/yl826/3DPathLoss/nc_raytracing/xml_to_heatmap/xml_to_heatmap_plotting/cm_npy_different_polarisations/'
sionna_cm_path_dict = {'E': sionna_cm_base + 'PCI=20,Rx_pol=H,Tx_pol=cross.npy', 
                       'F': sionna_cm_base + 'PCI=20,Rx_pol=H,Tx_pol=cross.npy', 
                       'C': sionna_cm_base + 'PCI=6,Rx_pol=H,Tx_pol=cross.npy', 
                       'D': sionna_cm_base + 'PCI=6,Rx_pol=H,Tx_pol=cross.npy',
                       'A': sionna_cm_base + 'PCI=3,Rx_pol=H,Tx_pol=cross.npy',
                       'B': sionna_cm_base + 'PCI=3,Rx_pol=H,Tx_pol=cross.npy'}

# lat lon pairs, antenna location

lat_lon_center_dict = {'C': (35.999186254897836, -78.9397858960648),
                            'D': (35.999186254897836, -78.9397858960648),
                            'A': (35.999186254897836, -78.9397858960648),
                            'B': (35.999186254897836, -78.9397858960648),
                            'E':  (36.00307128210248, -78.93706070613855),
                            'F':  (36.00307128210248, -78.93706070613855)}

lat_lon_top_left_dict = {'C': (36.001483241320656, -78.94273733701348),
                             'D': (36.001483241320656, -78.94273733701348),
                             'A': (36.001483241320656, -78.94273733701348),
                             'B': (36.001483241320656, -78.94273733701348),
                             'E': (36.005429554748495, -78.93998431793196), 
                             'F': (36.005429554748495, -78.93998431793196)}  # lat, lon pairs for top left corner



device_sql_dic = ["Samsung", "pixel", "rpi"]

def checkNmkdir(path):
    if not os.path.exists(path):
        # Create the directory if it doesn't exist
        os.makedirs(path)

pci_azimuth_dict = {
     'F': 216,
     'E': 216,
     'D': 30,
     'C': 30,
     'B': 212,
     'A': 212

}

measurement_data = pd.read_csv("../data/measurement/masked_cleaned_view.csv")



In [3]:





db_cache_res = {}

PCI_list = list(lat_lon_top_left_dict.keys())
random_index = 0

for device_name in list(device_sql_dic.keys()):
    artifact_dir = base_dir +"/" + device_name
    artifact_dir_2nd = artifact_dir + "/" + "2nd"
    artifact_dir_fitted = artifact_dir + "/" + "fitted"
    checkNmkdir(artifact_dir)
    checkNmkdir(artifact_dir_2nd)
    checkNmkdir(artifact_dir_fitted)

    
    for which_PCI in PCI_list:

        

        print('Random_index: %d, SS_NUM: %d, PCI: %s' % (random_index, SS_NUM, which_PCI))
        save_iso_output_name = 'PCI=%s_iso_UNet_out' % (which_PCI)

        def log(log_dict, commit_flag=False):
            if USE_WANDB:
                wandb.log(log_dict,commit=commit_flag) 

        
        df_SigCapDetails = measurement_data[(measurement_data['mPci'] == which_PCI) & (measurement_data['device'] == device_name)]
        print(df_SigCapDetails)


        if SHOW:
            print(df_SigCapDetails['latitude'][0:5])
            print(df_SigCapDetails['longitude'][0:5])
            print((df_SigCapDetails['rsrp'][::100]))

        log({"Total Measurement Points": len(df_SigCapDetails["latitude"])})
        # 3.Load data
        # first, get idx_uuid.npy (building map name)
        name = name_dict[which_PCI]

        # load building map and downsample
        building_height_ori = np.load(b_map_path_dict[which_PCI])
        building_height_ori = building_height_ori[0:512, 0:512]
        building_height_arr = building_height_ori[::4, ::4]


        name_splited = name.split("_")
        file_name_id_part = name_splited[0]  # idxuuid
        tx_x = 256
        tx_y = 256
        tx_position = [tx_x // 4, tx_y // 4]
        tx_height = 24
        # tx_height = np.max(building_height_arr)
        if SHOW:
            print('tx_height', int(tx_height))
        log({"tx_height": int(tx_height)})

        distance = np.arange(0, 1450, 1)

        # lookup table for path_loss_res, resolution=1 meter
        path_loss_res =  cu.pathloss_38901(distance,3.64, h_bs=int(tx_height), h_ut=2)

        # generate list of tuples of (numpy_row, numpy_col, rsrp) by converting lat,lon into 
        # local coordinates. 'list of tuples' is represented by np.array
        measurement = np.empty([len(df_SigCapDetails["latitude"]), 3])
        measurement[:, 0] = df_SigCapDetails["latitude"]
        measurement[:, 1] = df_SigCapDetails["longitude"]
        measurement[:, 2] = df_SigCapDetails["rsrp"]

        for idx, point in enumerate(measurement):
            row, col = cu.gps2local(measurement[idx,0], measurement[idx,1], 
                                    top_left_latitude=lat_lon_top_left_dict[which_PCI][0],
                                    top_left_longitude=lat_lon_top_left_dict[which_PCI][1])
            measurement[idx, 0] = row
            measurement[idx, 1] = col

        # get sparse_ss
        ground_truth_list, sparse_ss_arr = np.split(np.random.default_rng(seed=42).permutation(measurement), [len(measurement)-200])
        ground_truth_list = measurement

        sparse_ss_arr[:, 0] = sparse_ss_arr[:, 0]/4.0
        sparse_ss_arr[:, 1] = sparse_ss_arr[:, 1]/4.0

        increment_permutations = np.array(list(itertools.permutations([-1, 0, 1]))).reshape(-1, 1)

        def col_row_for_no_building(row_, col_, bh_arr=building_height_arr, 
                                    increment=increment_permutations):
            """
            :param row_: index that has height != 0
            :param col_: index that has height != 0
            :param bh_arr: building height array
            :param increment: tuples for incrementing [row,col]
            """
            for iii in range(3):
                for jjj in range(3):
                    try: 
                        height_ = bh_arr[row_+increment[2*iii][jjj], col_+increment[2*iii+1][jjj]]
                    except IndexError:
                        continue
                    if height_ == 0:
                        return row_+increment[2*iii][jjj], col_+increment[2*iii+1][jjj]
            # cannot find no building area in 3x3 region
            return None, None
        if SHOW:
            print("# of points for point-net: ", len(sparse_ss_arr))
            print("# of points for ground_truth: ", len(ground_truth_list))

        log({"# of points for S": len(sparse_ss_arr)})
        log({"# of points for GT": len(ground_truth_list)})

        ground_truth_tmp =  np.empty((128, 128), dtype=object)
        count_num_no_building = 0
        
        print(len(ground_truth_list))
        for point in ground_truth_list:
            row = int(point[0]/4)
            col = int(point[1]/4)
            if row >=128 or col >= 128 or row < 0 or col < 0:
                #print("{}:{}".format(row,col))
                continue
            #valid_num_mea_dic[device_name][which_PCI] = valid_num_mea_dic[device_name][which_PCI] +1
            if building_height_arr[row,col] != 0:
                row, col = col_row_for_no_building(row, col)  # attempts to update row, col 
                if row is None or col is None:  # didn't find good row, col
                    count_num_no_building += 1
                    continue
            if ground_truth_tmp[row, col] is None:
                ground_truth_tmp[row, col] = np.array(point[2])
            else:
                ground_truth_tmp[row][col] = np.append(ground_truth_tmp[row][col], point[2])
                
        if SHOW:
            print("number of points where there's a building", count_num_no_building)
        log({"number of points where there's a building": count_num_no_building})

        ground_truth_arr = np.empty((128, 128))
        for row in range(128):
            for col in range(128):
                if ground_truth_tmp[row, col] is None:
                    ground_truth_arr[row][col] = -160
                    
                else:
                    ground_truth_arr[row][col] = ground_truth_tmp[row,col].mean()
                    ground_truth_arr[row][col] = np.median(ground_truth_tmp[row,col])
                    ground_truth_arr[row][col] = 10 * np.log10( ( 10 ** ( ground_truth_tmp[row,col]/ 10)  ).mean() )


        
        print("Sanity Chrck")
        print(np.count_nonzero(ground_truth_arr != -160))
        if SHOW:
            plt.imshow(ground_truth_arr)
            plt.show()



    
        '''
        Random pick x points and then compute the mean of it
        '''      
        ss_map = np.full((128, 128), -160)
        tmp_ss_list = []
        count = 0
        while count < 100:
                    
            xx = random.randint(0, 127)
            yy = random.randint(0, 127)
            if ground_truth_arr[xx][yy] != -160 and ss_map[xx][yy] == -160:
                count+=1
                ss_map[xx][yy] = ground_truth_arr[xx][yy]
                tmp_ss_list.append(ground_truth_arr[xx][yy])
        GT_mean =  sum(tmp_ss_list) / len(tmp_ss_list)
        '''
        Generate the fitted Sionna Result
        '''  
        sionna_cm = 10 * np.log10(np.flip(np.load(sionna_cm_path_dict[which_PCI]),0))
        sionna_cm = np.where(building_height_arr!=0, -160, sionna_cm)
        sionna_cm = np.where(np.isinf(sionna_cm), -160, sionna_cm)
        

        np.save('%s/%s,RT.npy'% (artifact_dir, which_PCI),sionna_cm)
        np.save('%s/%s,RT.npy'% (artifact_dir_fitted, which_PCI),sionna_cm + (GT_mean -  np.mean(np.ma.masked_where(ground_truth_arr==-160, sionna_cm))))
        
        # Construct the TX position channel
        tx_position_channel = np.full((128, 128), 0, dtype=int)
        tx_position_channel[tx_position[1]][tx_position[0]] = tx_height


        path_loss_heat_map = np.full((128, 128), 0, dtype=float)



        for row in range(path_loss_heat_map.shape[0]):
            for col in range(path_loss_heat_map.shape[1]):
                # Compute the distance between pixel and tx
                dist = math.sqrt((tx_position[1]*4 - row*4)**2 + (tx_position[0]*4 - col*4)**2)
                path_loss_heat_map[row][col] =  -1 * path_loss_res[int(dist)]
        ### net for isotropic antenna
        np.save('%s/%s,ITM.npy'% (artifact_dir, which_PCI), path_loss_heat_map)
        np.save('%s/%s,ITM.npy'% (artifact_dir_fitted, which_PCI),path_loss_heat_map + (GT_mean -  np.mean(np.ma.masked_where(ground_truth_arr==-160, path_loss_heat_map))))
        

        net_iso = UNet(n_channels=2, n_classes=1, bilinear=False, pathloss=True)
        device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        net_iso.to(device=device)


        # 2.Load state dict
        #state_dict = torch.load(state_dict_1st_UNet, map_location=device)
        state_dict = torch.load(state_dict_1st_UNet, map_location=device)

        net_iso.load_state_dict(state_dict)


        # Since right now GT.size is 100*100 and other two size is 1000 * 1000, just check the input.
        # assert building_height_arr.shape == terrain_height_aimport loggin
        combined_input = np.zeros((4, 128, 128), dtype=float)

        # Combine all the channels together
        combined_input[0,:, :] = np.copy(building_height_arr)
        
        #
        combined_input[1,:, :] = tx_position_channel
        combined_input[2,:, :] = path_loss_heat_map


        batch = {
                    'combined_input': torch.as_tensor(combined_input.copy()).float().contiguous(),
                    'ground_truth': torch.as_tensor(ground_truth_arr.copy()).long().contiguous(),
                    'file_name': name,
                    'sparse_ss': torch.as_tensor(sparse_ss_arr.copy()).float().contiguous()
                }


        # Eval
        images, ground_truth = batch['combined_input'], batch['ground_truth']
        sparse_ss = batch['sparse_ss']

        # Since we directly load the data by manual, add batch dimension to the front of data
        images = images[None,:, :, :]
        sparse_ss = sparse_ss[None,:, :]


        net_iso.eval()

        images = images.to(device=device, dtype=torch.float32, memory_format=torch.channels_last)
        ground_truth = ground_truth.to(device=device, dtype=torch.long)
        sparse_ss = sparse_ss.to(device=device, dtype=torch.float32)

        with torch.no_grad():
            output = net_iso(images, sparse_ss).squeeze().cpu()

        output_iso_UNet = output.squeeze().cpu().numpy()
        #np.save('%s/%d,%d,%d,1stUNet.npy'% (artifact_dir, random_index, SS_NUM, which_PCI), output_iso_UNet)

        np.save('%s/%s,PLNet.npy'% (artifact_dir, which_PCI), output_iso_UNet)
        np.save('%s/%s,PLNet.npy'% (artifact_dir_fitted, which_PCI), output_iso_UNet + (GT_mean -  np.mean(np.ma.masked_where(ground_truth_arr==-160, output_iso_UNet))))

        if SHOW or True:
            fig, ax = plt.subplots(1, 1)
            
            image = ax.imshow(output_iso_UNet, interpolation='none', rasterized=True)

            ax.set_title('Iso UNet', size=22)
            plt.axis("off")
            fig.colorbar(image, ax=ax, orientation='vertical')
            fig.tight_layout()

            log({"1st UNet Predict":fig})
        plt.savefig('output_iso_UNet_PCI%s.pdf' % which_PCI, dpi=500, bbox_inches='tight')

        np.save(save_iso_output_name, output_iso_UNet)
        # UNet for tr38901

        # 1.Create the Model
        net = UNet(n_channels=3, n_classes=1, bilinear=False, pathloss=False)
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        net.to(device=device)


        # 2.Load state dict
        #state_dict = torch.load("../Pytorch-UNet-master/checkpoints_transfer/checkpoint_epoch21.pth", map_location=device)
        state_dict = torch.load(state_dict_2nd_UNet, map_location=device)

        net.load_state_dict(state_dict)




        # Since right now GT.size is 100*100 and other two size is 1000 * 1000, just check the input.
        # assert building_height_arr.shape == terrain_height_aimport loggin
        combined_input = np.zeros((3, 128, 128), dtype=float)

        # Combine all the channels together
        combined_input[0,:, :] = building_height_arr
        ss_map = np.full((128, 128), -160)

        res = []
        ss_map = np.full((128, 128), -160)
        count = 0
        while count < SS_NUM:
            
            xx = random.randint(0, 127)
            yy = random.randint(0, 127)
            if ground_truth_arr[xx][yy] != -160 and ss_map[xx][yy] == -160:
                count+=1
                ss_map[xx][yy] = ground_truth_arr[xx][yy]

        combined_input[1,:, :] = ss_map
        if SHOW:
            plt.imshow(ss_map)
            plt.colorbar()
            plt.close()
        combined_input[2,:, :] = np.load(save_iso_output_name + '.npy') 
        #combined_input[2,:, :] = output_iso_UNet

        batch = {
                    'combined_input': torch.as_tensor(combined_input.copy()).float().contiguous(),
                    'ground_truth': torch.as_tensor(ground_truth_arr.copy()).long().contiguous(),
                    'file_name': name,
                    'sparse_ss': torch.as_tensor(sparse_ss_arr.copy()).float().contiguous()
                }




        # 4. Eval
        images, ground_truth = batch['combined_input'], batch['ground_truth']
        sparse_ss = batch['sparse_ss']

        # Since we directly load the data by manual, add batch dimension to the front of data
        images = images[None,:, :, :]
        sparse_ss = sparse_ss[None,:, :]


        net.eval()

        images = images.to(device=device, dtype=torch.float32, memory_format=torch.channels_last)
        ground_truth = ground_truth.to(device=device, dtype=torch.long)
        sparse_ss = sparse_ss.to(device=device, dtype=torch.float32)

        with torch.no_grad():
            output = net(images, sparse_ss).cpu()

        output = output.squeeze().cpu().numpy()

        #np.save('%s/%d,2ndUNet.npy'% (artifact_dir, which_PCI), output)
        np.save('%s/%d,%d,%s,2ndUNet.npy'% (artifact_dir_2nd, 0, 200, which_PCI), output)




        def generate_pathloss_heatmap(model_name, h_bs, h_ut, freq, shape, tx_position):
            model_dict = {
            "ericsson": cu.pathloss_friis_free_space_model,
            "friis": cu.pathloss_ericsson_model,
            }
            
            func = model_dict.get(model_name, None)
            assert func is not None, "Model name not found! Try ericsson or friis"
            assert freq > 10, "Freq in MHz! Not GHz"
            
            
            path_loss_heat_map = np.full(shape, 0, dtype=float)
            
            for row in range(path_loss_heat_map.shape[0]):
                for col in range(path_loss_heat_map.shape[1]):
                    
                    dist = math.sqrt((tx_position[0]*4 - row*4)**2 + (tx_position[1]*4 - col*4)**2)
                    
                    path_loss_heat_map[row][col] =  -1 * func(dist/1000, h_bs, h_ut, freq )
            return path_loss_heat_map
        ericsson_pl = generate_pathloss_heatmap("ericsson",int(tx_height), 2, 3640,  (128,128), tx_position )

        if SHOW:
            plt.imshow(ericsson_pl)
            plt.title("Ericsson")
            plt.colorbar()
            plt.show()
        

        np.save('%s/%s,ericsson.npy'% (artifact_dir, which_PCI), ericsson_pl)
        
        np.save('%s/%s,ericsson.npy'% (artifact_dir_fitted, which_PCI),ericsson_pl + (GT_mean -  np.mean(np.ma.masked_where(ground_truth_arr==-160, ericsson_pl))))

        #generate_linear_regression_result(ericsson_pl , ground_truth_arr, "ori")
        # print(GT_mean )
        # print(np.ma.masked_where(building_height_arr!=0, ericsson_pl))
        # plt.imshow(np.ma.masked_where(building_height_arr!=0, ericsson_pl))
        # plt.show()
        # print(np.mean(np.ma.masked_where(building_height_arr!=0, ericsson_pl)))
        #generate_linear_regression_result(ericsson_pl + (GT_mean -  np.mean(np.ma.masked_where(building_height_arr!=0, ericsson_pl))), ground_truth_arr, "fitted")


        friis_pl = generate_pathloss_heatmap("friis",int(tx_height), 2, 3640, (128,128), tx_position )
    
        np.save('%s/%s,friis.npy'% (artifact_dir, which_PCI), friis_pl)
        np.save('%s/%s,friis.npy'% (artifact_dir_fitted, which_PCI),friis_pl + (GT_mean -  np.mean(np.ma.masked_where(ground_truth_arr==-160, friis_pl))))
        if SHOW:
            plt.imshow(friis_pl)
            plt.title("Friis")
            plt.colorbar()
            plt.show()
        if SHOW:
            ericsson_coef, ericsson_intercept, ericsson_ori_rmse, ericsson_fitted_rmse = cu.generate_linear_regression_result(ericsson_pl,ground_truth_arr,"Ericsson")

            log({"Ericsson RMSE":ericsson_ori_rmse})

            friis_coef, friis_intercept, friis_ori_rmse, friis_fitted_rmse = cu.generate_linear_regression_result(friis_pl,ground_truth_arr,"friis")
            log({"friis RMSE":friis_ori_rmse})



        def generate_linear_regression_result(prediction, ground_truth, title="", print_result=True,SHOW=False):
            """
            :param prediction:  predicted value
            :param ground_truth: ground truth value
            :param title:
            :param print_result:
            :return:
            """
            assert ground_truth.shape == prediction.shape, "ground_truth and prediction shape mismatch "
        
            print()
            print("++++++++++++++", title, "++++++++++++++")
            X = []
            y = []
            for row in range(prediction.shape[0]):
                for col in range(prediction.shape[1]):
                    if ground_truth[row, col] != -160:
                        X.append(prediction[row, col])
                        y.append(ground_truth[row, col])
            X = np.array(X).reshape(-1, 1)
            tmp_res = []
            tmp_x = np.arange(0.05,1.0,0.05)
            ori_rmse = mean_squared_error(y, X, squared=False)
        
            print("original without fitting rmse ", ori_rmse)
            log({title + " RMSE":ori_rmse})

            for i in np.arange(0.05,1.0,0.05):
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = i, random_state = 42)
                reg = LinearRegression().fit(X_train, y_train)

        #         print("score", reg.score(X, y))
        #         print("coef", reg.coef_[0])

        #         print("intercept", reg.intercept_)

                ori_rmse = mean_squared_error(y, X, squared=False)
                

                fitted_rmse = mean_squared_error(y, X * reg.coef_ + reg.intercept_, squared=False)
                #print("rmse after linear regression ", fitted_rmse)

                ori_mae = mean_absolute_error(y, X)
                fitted_mae = mean_absolute_error(y, X * reg.coef_ + reg.intercept_)
                # print("mae ", ori_mae)
                # print("mae after linear regression ", fitted_rmse)
                tmp_res.append(fitted_rmse)
            if SHOW:
                plt.plot(tmp_x, tmp_res)
                plt.title("Test set ratio vs. RMSE")
                plt.xlabel('Test set ratio')
                plt.ylabel('RMSE in dB')

                plt.show()
            
            return reg.coef_[0], reg.intercept_, ori_rmse, fitted_mae
            

        if SHOW:
            ml_coef, ml_intercept, ml_ori_rmse, ml_fitted_rmse = generate_linear_regression_result(output,ground_truth_arr,"ML")
            pl_coef, pl_intercept, pl_ori_rmse, pl_fitted_rmse = generate_linear_regression_result(path_loss_heat_map,ground_truth_arr,"PL")
            fig, ax = plt.subplots(1, 1)
            image = ax.imshow(output, interpolation='none', rasterized=True)

            ax.set_title('Directional UNet', size=22)
            plt.axis("off")
            fig.colorbar(image, ax=ax, orientation='vertical')
            fig.tight_layout()
            log({"2nd UNet Predict":fig})
        # plt.savefig('output_directional_UNet_PCI%d.pdf' % which_PCI, dpi=500, bbox_inches='tight')
        # base_path_sionna = '/home/yl826/3DPathLoss/nc_raytracing/'
        # sionna_cm_path_dict = {20: 'cm_plane_duke_pci20_7e6_tr38901Tx/'
        #                        }
        # f_names_sig_map_idx_uuid = list(set([f.split('_')[0] + '_' + f.split('_')[1]
        #                                     for f in os.listdir(base_path_sionna + 
        #                                                         sionna_cm_path_dict[which_PCI])
        #                                 if os.path.isfile(base_path_sionna + 
        #                                                   sionna_cm_path_dict[which_PCI] + f)]))
        # f_names = list(set([f for f in os.listdir(base_path_sionna + sionna_cm_path_dict[which_PCI])
        #                                 if os.path.isfile(base_path_sionna + 
        #                                                   sionna_cm_path_dict[which_PCI] + f)]))
        # assert len(f_names) == 1
        # sionna_cm = 10 * np.log10(np.load(base_path_sionna + sionna_cm_path_dict[which_PCI] + f_names[0]))
        # sionna_cm = np.nan_to_num(sionna_cm, copy=True, nan=-160, posinf=-160, neginf=-160)
        # sionna_cm[sionna_cm < -160] = -160
        # fig, ax = plt.subplots(1, 1)
        # image = ax.imshow(sionna_cm, interpolation='none', rasterized=True)

        # ax.set_title('Coverage Map', size=22)
        # plt.axis("off")
        # fig.colorbar(image, ax=ax, orientation='vertical')
        # fig.tight_layout()
        # plt.savefig('coverage_map_PCI%d.pdf' % which_PCI, dpi=500, bbox_inches='tight')
        
        
        
        
        if SHOW:
            b_map_npy = np.load(b_map_path_dict[which_PCI])[0:512, 0:512]

            fig, ax = plt.subplots(1, 1)
            image = ax.imshow(b_map_npy, interpolation='none', rasterized=True)

            ax.set_title('Building Map', size=22)
            fig.colorbar(image, ax=ax, orientation='vertical')
            plt.axis("off")
            fig.tight_layout()
            log({"Building Map":fig})
            #plt.savefig('building_map_PCI%d.pdf' % which_PCI, dpi=500, bbox_inches='tight')
            fig, ax = plt.subplots(1, 1)
            image = ax.imshow(ss_map, interpolation='none', rasterized=True)

            ax.set_title('Sparse Signal Map', size=22)
            plt.axis("off")
            fig.colorbar(image, ax=ax, orientation='vertical')
            fig.tight_layout()
            log({"Sparse Signal Map":fig})
            #plt.savefig('ss_map_PCI%d.pdf' % which_PCI, dpi=500, bbox_inches='tight')
            fig, ax = plt.subplots(1, 1)
            image = ax.imshow(ground_truth_arr, interpolation='none', rasterized=True)

            ax.set_title('Ground Truth Measurements', size=22)
            plt.axis("off")
            fig.colorbar(image, ax=ax, orientation='vertical')
            fig.tight_layout()
            log({"Ground Truth Measurements":fig})
            #plt.savefig('ground_truth_map_PCI%d.pdf' % which_PCI, dpi=500, bbox_inches='tight')
            # cu.gps2local(36.00247804455967, -78.94120458940182)

            output_tmp = np.copy(output)
            output_mask = (ground_truth_arr == -160)
            result = ground_truth_arr - output_tmp
            result[output_mask] = np.nan
            fig, ax = plt.subplots(1, 1)
            image = ax.imshow(result , interpolation='none', rasterized=True)

            ax.set_title('Ground Truth - 2nd UNet Predict ', size=22)
            plt.axis("off")
            fig.colorbar(image, ax=ax, orientation='vertical')
            fig.tight_layout()
            log({"Ground Truth - 2nd UNet Predict":fig},True)

AttributeError: 'list' object has no attribute 'keys'